In [ ]:
%matplotlib inline

In [ ]:
#@title
import jax
import numpy as np

import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

try:
  import flax
except ModuleNotFoundError:
  !pip install --quiet flax==0.6.4
  import flax

try:
  import optax
except ModuleNotFoundError:
  !pip install --quiet optax
  import optax

from flax import linen as nn
from flax.training import train_state

from tqdm import trange
from functools import partial
from matplotlib import pyplot as plt

# Variational Auto-Encoder (VAE)

Remind that VAE minimizes the following variational bound
$$\text{KL}(p_{\text{data}}(x),p(x|\theta)) \leq \text{KL}(p_{\text{data}}(x)q(z|x;\eta),p(x|z;\theta)p(z|\theta))$$
i.e. 
$$\theta^*,\eta^* = \argmin_{\theta,\eta} \text{KL}(p_{\text{data}}(x)q(z|x;\eta),p(x|z;\theta)p(z|\theta)).$$
Eliminating the terms that don't depend on $\theta,\eta$ and rearranging them, we get
$$\text{Loss}(\eta,\theta) = -\mathbb{E}_{p_{\text{data}}(x)}\mathbb{E}_{q(z|x;\eta)} \log p(x|z;\theta) + \mathbb{E}_{p_{\text{data}}(x)}\text{KL}(q(z|x;\eta), p(z|\theta)).$$
In particular, here we consider the following parameterization.
The approximate posterior is parameterised as a normal distribution
$$q(z|x;\eta) = \mathcal{N}(z|\mu(x;\eta),\sigma(x;\eta)^2),$$
which parameters $\mu(x;\eta),\sigma(x;\eta)$ are defined by a neural network called "encoder".
The join likelihood is defined by another normal distribution
$$p(x|z;\theta) = \mathcal{N}(x|\mu(z;\theta),\sigma(z;\theta)^2),$$
where $\mu(z;\theta),\sigma(z;\theta)$ is defined by the "decoder", and the marginal distribution of latent variables is the standard normal gaussian, i.e.
$$p(z|\theta) = \mathcal{N}(z|0,1).$$

## Generate the data

In [ ]:
from typing import NamedTuple, Any

def sample_data(key, bs):
  keys = random.split(key, 3)
  label = random.randint(keys[0], minval=0, maxval=2, shape=(bs, 2))
  x_1 = 3*(label.astype(jnp.float32)-0.5)
  x_1 += 4e-1*random.normal(keys[1], shape=(bs,2))
  return x_1, label
seed = 0
np.random.seed(seed)
key = random.PRNGKey(seed)
bs = 512

key, ikey = random.split(key)
data, labels = sample_data(ikey, 1024)
plt.scatter(data[:,0], data[:,1], alpha=0.3, label='data')
plt.legend()
plt.grid()

## Define the Model

For parameterizing both encoder and decoder, we are going to use an MLP that has two outputs: one to parameterize the mean and another to parameterize the variance.

In [ ]:
class MLP(nn.Module):
  num_hid : int
  num_out : int

  @nn.compact
  def __call__(self, x):
    h = x
    h = nn.Dense(features=self.num_hid)(h)
    h = nn.relu(h)
    h = nn.Dense(features=self.num_hid)(h)
    h = nn.swish(h)
    h = nn.Dense(features=self.num_hid)(h)
    h = nn.swish(h)
    h = nn.Dense(features=2*self.num_out)(h)
    return h[:,:self.num_out], h[:,self.num_out:]

In [ ]:
latent_dim = 2
encoder = MLP(num_hid=512, num_out=latent_dim)
decoder = MLP(num_hid=512, num_out=data.shape[1])
print(encoder, decoder)
key, *init_key = random.split(key, 3)
state_enc = train_state.TrainState.create(apply_fn=encoder.apply,
                                          params=encoder.init(init_key[0], data),
                                          tx=optax.adam(learning_rate=2e-4))
state_dec = train_state.TrainState.create(apply_fn=decoder.apply,
                                          params=decoder.init(init_key[1], jnp.ones((data.shape[0], latent_dim))),
                                          tx=optax.adam(learning_rate=2e-4))

## Loss Function (5 points)
In general, the loss looks like this
$$\text{Loss}(\eta,\theta) = -\mathbb{E}_{p_{\text{data}}(x)}\mathbb{E}_{q(z|x;\eta)} \log p(x|z;\theta) + \mathbb{E}_{p_{\text{data}}(x)}\text{KL}(q(z|x;\eta), p(z|\theta))$$
- Derive the loss for the encoder, decoder, prior parameterized as Gaussians (as described above). Note that you have to find the best estimation, i.e. reduce the objective to the minimum number of Monte Carlo estimates possible. 
- Implement the objective in the function below.
- Train the model with the derived objective. You can generate new batch at each step using the 'sample_data' function. Does the loss curve look meaningful?

In [ ]:
def vae_objective(states, key, params, bs):
  # YOUR CODE HERE
  print(loss.shape, 'final.shape', flush=True)
  return loss.mean()

@partial(jax.jit, static_argnums=2)
def train_step(state_enc, state_dec, bs, key):
  grad_fn = jax.value_and_grad(vae_objective, argnums=2)
  loss, grads = grad_fn((state_enc, state_dec), key, (state_enc.params, state_dec.params), bs)
  state_enc = state_enc.apply_gradients(grads=grads[0])
  state_dec = state_dec.apply_gradients(grads=grads[1])
  return state_enc, state_dec, loss

key, loc_key = random.split(key)
state_enc, state_dec, loss = train_step(state_enc, state_dec, bs, loc_key)

## Train Loop

In [ ]:
num_iterations = 10_000

loss_plot = np.zeros(num_iterations)
for iter in trange(num_iterations):
  key, loc_key = random.split(key)
  state_enc, state_dec, loss = train_step(state_enc, state_dec, bs, loc_key)
  loss_plot[iter] = loss

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(loss_plot)
plt.xlabel('iterations')
plt.ylabel('loss')
plt.grid()

## Visualization of the Trained Model (5 points for both cells)
For the trained VAE
- Visualize generated samples using $p(x|z;\theta)p(z|\theta)$ and compare them to the samples from the data. Do they match?
- Visualize the latent distribution by sampling from $q(z|x;\eta)p_{\text{data}}(x)$ and compare them to the samples from $p(z|\theta)$. Do they match?

In [ ]:
key, *loc_key = random.split(key,5)
# YOUR CODE HERE

plt.figure(figsize=(12,6))
plt.subplot(121)
plt.scatter(# YOUR CODE HERE,
            alpha=0.3, label='data')
plt.scatter(# YOUR CODE HERE, 
            alpha=0.3, label='generations')
plt.xlim(-4,4)
plt.ylim(-4,4)
plt.legend()
plt.grid()

# YOUR CODE HERE
plt.subplot(122)
plt.scatter(# YOUR CODE HERE,
            alpha=0.3, label='prior')
plt.scatter(# YOUR CODE HERE,
            alpha=0.3, label='encoded data')
plt.xlim(-4,4)
plt.ylim(-4,4)
plt.legend()
plt.grid()

- Visualize how the latent space is divided between the modes. There is more than one way to visualize this. For instance, use the variable labels produced by 'sample_data', encode the data and color the samples. 

In [ ]:
# YOUR CODE HERE

plt.figure(figsize=(12,6))
plt.subplot(121)
for # YOUR CODE HERE
  plt.scatter(# YOUR CODE HERE,
              alpha=0.3)
plt.xlim(-4,4)
plt.ylim(-4,4)
plt.grid()

plt.subplot(122)
for # YOUR CODE HERE
  plt.scatter(# YOUR CODE HERE, 
              alpha=0.3)
plt.xlim(-4,4)
plt.ylim(-4,4)
plt.grid()

## Evaluation of the trained model (5 points)

Let's evaluate the model by estimating $\mathbb{E}_{p_{\text{data}}(x)}\log p(x|\theta)$. Namely, consider three different estimates:
1. Naive marginal likelihood estimation 
$$\mathbb{E}_{p_{\text{data}}(x)}\log p(x|\theta) = \mathbb{E}_{p_{\text{data}}(x)}\log \int dz\;p(x|z;\theta)p(z|\theta)$$
2. VAE lower bound
$$\mathbb{E}_{p_{\text{data}}(x)}\log p(x|\theta) \geq \mathbb{E}_{p_{\text{data}}(x)} \mathbb{E}_{q(z|x;\eta)}\log \frac{p(x|z;\theta)p(z|\theta)}{q(z|x;\eta)} \approx \mathbb{E}_{p_{\text{data}}(x)} \frac{1}{K}\sum_{k=1}^K\log \frac{p(x|z_k;\theta)p(z_k|\theta)}{q(z_k|x;\eta)}, \;\; z_k \sim q(z|x;\eta)$$
3. IWAE lower bound
$$\mathbb{E}_{p_{\text{data}}(x)}\log p(x|\theta) \geq \mathbb{E}_{p_{\text{data}}(x)}\mathbb{E}_{z_1,\ldots, z_k \sim q(z|x;\eta)}\log \frac{1}{K}\sum_{k=1}^K\frac{p(x|z_k;\theta)p(z_k|\theta)}{q(z_k|x;\eta)}$$

Implement the functions below and compare them by plotting the log-likelihood estimates with the number of samples and estimating the density on the state-space. Which estimate is better?

In [ ]:
def marginal_likelihood(key, x, n_samples, state_enc, state_dec):
  # YOUR CODE HERE

def vae_bound(key, x, n_samples, state_enc, state_dec):
  # YOUR CODE HERE

def iwae_bound(key, x, n_samples, state_enc, state_dec):
  # YOUR CODE HERE

In [ ]:
key, *ikey = random.split(key, 3)
print('vae bound:', vae_bound(ikey[1], data, 1000, state_enc, state_dec).mean())
print('iwae bound:', iwae_bound(ikey[1], data, 1000, state_enc, state_dec).mean())

n_samples = [1,10,100,1000]
ll_iwae = []
ll_vae = []
for n in n_samples:
  ll_vae.append(vae_bound(ikey[1], data, n, state_enc, state_dec).mean())
  ll_iwae.append(iwae_bound(ikey[1], data, n, state_enc, state_dec).mean())
  
plt.plot(n_samples, ll_vae)
plt.plot(n_samples, ll_iwae)
plt.xscale('log')
plt.xlabel('number of samples')
plt.ylabel('likelihood estimate')
plt.grid()

In [ ]:
key, *ikey = random.split(key, 4)
x,y = jnp.linspace(-4,4,50), jnp.linspace(-4,4,50)
x,y = jnp.meshgrid(x,y)
grid = jnp.stack([x, y]).transpose((1,2,0)).reshape((-1,2))

for n_samples in [1,10,100,1000]:
  plt.figure(figsize=(16,5))
  
  ll = marginal_likelihood(ikey[0], grid, n_samples, state_enc, state_dec)
  plt.subplot(131)
  plt.contourf(x,y,jnp.exp(ll).reshape(x.shape), levels=50)
  
  ll = vae_bound(ikey[1], grid, n_samples, state_enc, state_dec)
  plt.subplot(132)
  plt.contourf(x,y,jnp.exp(ll).reshape(x.shape), levels=50)

  ll = iwae_bound(ikey[2], grid, n_samples, state_enc, state_dec)
  plt.subplot(133)
  plt.contourf(x,y,jnp.exp(ll).reshape(x.shape), levels=50)
  plt.show()

## Importance Weighted Autoencoder (IWAE)

Why don't we try learning the entire model using IWAE?

In [ ]:
latent_dim = 2
encoder = MLP(num_hid=512, num_out=latent_dim)
decoder = MLP(num_hid=512, num_out=data.shape[1])
print(encoder, decoder)
key, *init_key = random.split(key, 3)
state_enc = train_state.TrainState.create(apply_fn=encoder.apply,
                                          params=encoder.init(init_key[0], data),
                                          tx=optax.adam(learning_rate=2e-4))
state_dec = train_state.TrainState.create(apply_fn=decoder.apply,
                                          params=decoder.init(init_key[1], jnp.ones((data.shape[0], latent_dim))),
                                          tx=optax.adam(learning_rate=2e-4))

## IWAE objective (5 points)

- Implement the IWAE objective, i.e. 
$$\theta^*,\eta^* = \argmin_{\theta,\eta}-\mathbb{E}_{p_{\text{data}}(x)}\mathbb{E}_{z_1,\ldots, z_k \sim q(z|x;\eta)}\log \frac{1}{K}\sum_{k=1}^K\frac{p(x|z_k;\theta)p(z_k|\theta)}{q(z_k|x;\eta)}$$
- Train the model using the IWAE objective and $K=5$ at every iteration. How does the loss function behave?

In [ ]:
def iwae_objective(states, key, params, bs):
  # YOUR CODE HERE
  print(loss.shape, 'final.shape', flush=True)
  return loss.mean()

@partial(jax.jit, static_argnums=2)
def train_step(state_enc, state_dec, bs, key):
  grad_fn = jax.value_and_grad(iwae_objective, argnums=2)
  loss, grads = grad_fn((state_enc, state_dec), key, (state_enc.params, state_dec.params), bs)
  state_enc = state_enc.apply_gradients(grads=grads[0])
  state_dec = state_dec.apply_gradients(grads=grads[1])
  return state_enc, state_dec, loss

key, loc_key = random.split(key)
state_enc, state_dec, loss = train_step(state_enc, state_dec, bs, loc_key)

In [ ]:
num_iterations = 10_000

loss_plot = np.zeros(num_iterations)
for iter in trange(num_iterations):
  key, loc_key = random.split(key)
  state_enc, state_dec, loss = train_step(state_enc, state_dec, bs, loc_key)
  loss_plot[iter] = loss

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(loss_plot)
plt.xlabel('iterations')
plt.ylabel('loss')
plt.grid()

## Analysis of IWAE (3 points)

- Reproduce the visualizations that you did for VAE for the IWAE.
- Reproduce the plots for the density and log-likelihood estimation.
- How can we compare IWAE and VAE? Which model is better? Why?

In [ ]:
# YOUR CODE HERE
plt.figure(figsize=(12,6))
plt.subplot(121)
plt.scatter(# YOUR CODE HERE,
            alpha=0.3, label='data')
plt.scatter(# YOUR CODE HERE,
            alpha=0.3, label='generations')
plt.xlim(-4,4)
plt.ylim(-4,4)
plt.legend()
plt.grid()

# YOUR CODE HERE
plt.subplot(122)
plt.scatter(# YOUR CODE HERE,
            alpha=0.3, label='prior')
plt.scatter(# YOUR CODE HERE,
            alpha=0.3, label='encoded data')
plt.xlim(-4,4)
plt.ylim(-4,4)
plt.legend()
plt.grid()

In [ ]:
# YOUR CODE HERE
plt.figure(figsize=(12,6))
plt.subplot(121)
for # YOUR CODE HERE
  plt.scatter(# YOUR CODE HERE, 
              alpha=0.3)
plt.xlim(-4,4)
plt.ylim(-4,4)
plt.grid()

plt.subplot(122)
for # YOUR CODE HERE
  plt.scatter(# YOUR CODE HERE, 
              alpha=0.3)
plt.xlim(-4,4)
plt.ylim(-4,4)
plt.grid()

In [ ]:
key, *ikey = random.split(key, 4)
x,y = jnp.linspace(-4,4,50), jnp.linspace(-4,4,50)
x,y = jnp.meshgrid(x,y)
grid = jnp.stack([x, y]).transpose((1,2,0)).reshape((-1,2))

for n_samples in [1,10,100,1000]:
  plt.figure(figsize=(16,5))
  
  ll = marginal_likelihood(ikey[0], grid, n_samples, state_enc, state_dec)
  plt.subplot(131)
  plt.contourf(x,y,jnp.exp(ll).reshape(x.shape), levels=50)
  
  ll = vae_bound(ikey[1], grid, n_samples, state_enc, state_dec)
  plt.subplot(132)
  plt.contourf(x,y,jnp.exp(ll).reshape(x.shape), levels=50)

  ll = iwae_bound(ikey[2], grid, n_samples, state_enc, state_dec)
  plt.subplot(133)
  plt.contourf(x,y,jnp.exp(ll).reshape(x.shape), levels=50)
  plt.show()

In [ ]:
key, *ikey = random.split(key, 3)
print('vae bound:', vae_bound(ikey[1], data, 1000, state_enc, state_dec).mean())
print('iwae bound:', iwae_bound(ikey[1], data, 1000, state_enc, state_dec).mean())

n_samples = [1,10,100,1000]
ll_iwae_trained = []
for n in n_samples:
  ll_iwae_trained.append(iwae_bound(ikey[1], data, n, state_enc, state_dec).mean())
  
plt.plot(n_samples, ll_iwae)
plt.plot(n_samples, ll_iwae_trained)
plt.xscale('log')
plt.xlabel('number of samples')
plt.ylabel('likelihood estimate')
plt.grid()

## Next steps? (Open-ended questions, 2 points)
- What are the next steps for exploring VAE and IWAE?
- Can you come up with a 2d dataset that highlights their differences?
- What happens if we change the dimensionality of the latent space?